# Applying Advanced Transformations
Cameron Peace


## Task

<mark><u>**I. Clean the files and combine them into one final DataFrame.**

This dataframe should have the following columns:

* [ ] Hero (Just the name of the Hero)
* [ ] Publisher
* [ ] Gender
* [ ] Eye color
* [ ] Race
* [ ] Hair color
* [ ] Height (numeric)
* [ ] Skin color
* [ ] Alignment
* [ ] Weight (numeric)
* [ ] Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
    * Agility
    * Flight
    * Superspeed
    * etc.

Hint: There is a space in "100 kg" or "52.5 cm"

<mark><u>**II. Use your combined DataFrame to answer the following questions.**

* [ ] Compare the average weight of super powers who have Super Speed to those who do not.
* [ ] What is the average height of heroes for each publisher?

## Data Dictionary

* **Row Identification** - index
* **name** 
* **Gender** 
* **Eye color** 
* **Race** 
* **Hair color** 
* **Height** - The height measured in centimeters
* **Publisher** - comics publisher
* **Skin color**
* **Alignment** - (good, bad, neutral, unknown)
* **Weight** - The weight measured in pounds

### Data Background

This dataset is a modified version of a dataset found [here](https://www.kaggle.com/datasets/claudiodavi/superhero-set) on kaggle.  It has been altered by Coding Dojo for educational purposes.

## Imports, Loading, Viewing Data

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [4]:
# loading the data
df_info = pd.read_csv('superhero_info.csv')
df_powers = pd.read_csv('superhero_powers.csv')

# making a copy
df_info_original = df_info.copy()
df_powers_original = df_powers.copy()

#### Viewing Superhero info df

In [8]:
# viewing superhero info
display(df_info.head())
display(df_info.info(), df_info.describe(), df_info.columns)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


None

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
count,463,463,463,463,463,463,463,463
unique,457,3,52,4,29,21,14,287
top,Spider-Man|Marvel Comics,Male,Human,good,Black,blue,Unknown,"{'Height': '183.0 cm', 'Weight': '90.0 kg'}"
freq,3,321,156,311,127,182,409,6


Index(['Hero|Publisher', 'Gender', 'Race', 'Alignment', 'Hair color',
       'Eye color', 'Skin color', 'Measurements'],
      dtype='object')

#### Viewing Superhero powers df

In [7]:
# viewing the data
display(df_powers.head())
display(df_powers.info(), df_powers.describe(), df_powers.columns)

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


None

,hero_names,Powers
count,667,667
unique,667,608
top,3-D Man,Intelligence
freq,1,8


Index(['hero_names', 'Powers'], dtype='object')

<mark><u>**Comment:**

* <font color='dodgerblue' size=4><i>
We'll have to split the name/publisher and join with the other df on name here.  I'm going to do this first so I can consolidate into one df.
* <font color='dodgerblue' size=4><i> 
We have a little over 200 more entries for the hero powers dataset than the primary dataset.  I think for our purposes it makes sense to drop any superheros present in the 'powers' dataset that we don't have other information on.  I might consider doing a outer join just to view what we're missing from each.
<font color='dodgerblue' size=4>


## Cleaning

### Splitting name/publisher to two columns

In [12]:
# making the new columns
df_info[['name', 'publisher']] = df_info['Hero|Publisher'].str.split('|', expand=True)

# confirmings
df_info[['name', 'publisher', 'Hero|Publisher']]

,name,publisher,Hero|Publisher
0,A-Bomb,Marvel Comics,A-Bomb|Marvel Comics
1,Abe Sapien,Dark Horse Comics,Abe Sapien|Dark Horse Comics
2,Abin Sur,DC Comics,Abin Sur|DC Comics
3,Abomination,Marvel Comics,Abomination|Marvel Comics
4,Absorbing Man,Marvel Comics,Absorbing Man|Marvel Comics
...,...,...,...
458,Yellowjacket,Marvel Comics,Yellowjacket|Marvel Comics
459,Yellowjacket II,Marvel Comics,Yellowjacket II|Marvel Comics
460,Yoda,George Lucas,Yoda|George Lucas
461,Zatanna,DC Comics,Zatanna|DC Comics


<mark><u>**Comment:**</u>

<font color='dodgerblue' size=4><i>
Looks good, we can now drop the combined column ('Hero/Publisher')
</i></font>

In [14]:
# dropping 'Hero/Publisher column'
df_info = df_info.drop(columns='Hero|Publisher').copy()

# confirming
df_info.columns

Index(['Gender', 'Race', 'Alignment', 'Hair color', 'Eye color', 'Skin color',
       'Measurements', 'name', 'publisher'],
      dtype='object')

### Joining Dataframes

In [15]:
# joining with a outer join to see where we are missing data from df_info
df_all = df_info.merge(df_powers, left_on='name', right_on='hero_names', how='outer')

In [21]:
# viewing
display(df_all.sample(5))

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,name,publisher,hero_names,Powers
534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dormammu,"Power Augmentation,Super Strength,Telepathy,Si..."
317,Female,Human,bad,Red,green,green,"{'Height': '168.0 cm', 'Weight': '50.0 kg'}",Poison Ivy,DC Comics,Poison Ivy,"Agility,Accelerated Healing,Marksmanship,Super..."
530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data,"Agility,Cold Resistance,Durability,Longevity,I..."
53,Female,New God,bad,Black,blue,Unknown,"{'Height': '188.0 cm', 'Weight': '135.0 kg'}",Big Barda,DC Comics,Big Barda,"Agility,Durability,Flight,Longevity,Super Stre..."
389,Female,Unknown,bad,White,red,Unknown,"{'Height': '178.0 cm', 'Weight': '58.0 kg'}",Spider-Woman IV,Marvel Comics,Spider-Woman IV,"Agility,Flight,Super Strength,Telepathy,Energy..."


<mark><u>**Comment:**</u>

<font color='dodgerblue' size=4><i>
After reviewing the results of the full join in greater depth, it does not appear that we have dirty data that would result in superheros dropped which should be added to our final dataset.  There are simply superheros in the 'powers' dataset that do not appear in the 'info' set.  It makes sense to just do an inner join and lose the superheros for which we do not have information, especially considering that we are trying to answer a question regarding average weight.
</i></font>

In [33]:
# joining dfs
df = df_info.merge(df_powers, left_on='name', right_on='hero_names')

# making a copy for comparison if needed
df_original = df.copy()

In [35]:
# viewing the new df
display(df.info(), df.sample(3))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463 entries, 0 to 462
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Gender        463 non-null    object
 1   Race          463 non-null    object
 2   Alignment     463 non-null    object
 3   Hair color    463 non-null    object
 4   Eye color     463 non-null    object
 5   Skin color    463 non-null    object
 6   Measurements  463 non-null    object
 7   name          463 non-null    object
 8   publisher     463 non-null    object
 9   hero_names    463 non-null    object
 10  Powers        463 non-null    object
dtypes: object(11)
memory usage: 43.4+ KB


None

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,name,publisher,hero_names,Powers
292,Male,Human,bad,No Hair,brown,Unknown,"{'Height': '180.0 cm', 'Weight': '79.0 kg'}",Mysterio,Marvel Comics,Mysterio,"Weapons Master,Intelligence,Stamina,Weapon-bas..."
253,Male,Human,good,Black,brown,Unknown,"{'Height': '198.0 cm', 'Weight': '191.0 kg'}",Luke Cage,Marvel Comics,Luke Cage,"Accelerated Healing,Durability,Super Strength,..."
296,Female,Unknown,good,Blond,blue,Unknown,"{'Height': '168.0 cm', 'Weight': '101.0 kg'}",Namorita,Marvel Comics,Namorita,"Agility,Accelerated Healing,Durability,Flight,..."


<mark><u>**Comment:**</u>

<font color='dodgerblue' size=4><i>
Our join looks good, we can now drop 'hero_names', as it's redundant.  We are now working with 463 heroes (the heroes that we have more extensive info on, from df_info dataframe) 
</i></font>

In [36]:
# dropping 'hero_names'
df = df.drop(columns='hero_names').copy()

# confirming
df.columns

### Fixing Column Names

In [38]:
# changing column names to snakecase
df = df.rename(columns = lambda x: '_'.join(x.lower().split()))

# confirming
df.columns

Index(['gender', 'race', 'alignment', 'hair_color', 'eye_color', 'skin_color',
       'measurements', 'name', 'publisher', 'powers'],
      dtype='object')